In [1]:
import pyspark
from pyspark.sql import SQLContext

sc = pyspark.SparkContext('local[*]')
sqlc = SQLContext(sc)

In [2]:
data = '''balance,gender,user_id
10,,0
1,M,1
-0.5,F,2
0,F,3
5,,4
3,M,5
'''
with open('user_demo.csv', 'w') as output:
    output.write(data)

In [3]:
df = sqlc.read.format('com.databricks.spark.csv').options(
    header='true', inferschema='true').load(
        'file:///D:/Projects/python_projects/big_data/user_demo.csv')
df.show()

+-------+------+-------+
|balance|gender|user_id|
+-------+------+-------+
|   10.0|  null|      0|
|    1.0|     M|      1|
|   -0.5|     F|      2|
|    0.0|     F|      3|
|    5.0|  null|      4|
|    3.0|     M|      5|
+-------+------+-------+



In [4]:
df.printSchema()

root
 |-- balance: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- user_id: integer (nullable = true)



In [5]:
df

DataFrame[balance: double, gender: string, user_id: int]

In [6]:
df.dtypes

[('balance', 'double'), ('gender', 'string'), ('user_id', 'int')]

In [7]:
df.columns

['balance', 'gender', 'user_id']

In [8]:
df.filter(df['gender'] != 'null').filter(df['balance'] > 0).select(
    ['balance', 'gender', 'user_id']).show()

+-------+------+-------+
|balance|gender|user_id|
+-------+------+-------+
|    1.0|     M|      1|
|    3.0|     M|      5|
+-------+------+-------+



In [9]:
df.filter('gender is not null').filter('balance > 0').select(
    '*').show()  # 支持SQL字符串

+-------+------+-------+
|balance|gender|user_id|
+-------+------+-------+
|    1.0|     M|      1|
|    3.0|     M|      5|
+-------+------+-------+



In [10]:
df.filter('gender is not null and balance < 0').show()

+-------+------+-------+
|balance|gender|user_id|
+-------+------+-------+
|   -0.5|     F|      2|
+-------+------+-------+



In [11]:
# 处理缺失数据
df.na.drop().show()

+-------+------+-------+
|balance|gender|user_id|
+-------+------+-------+
|    1.0|     M|      1|
|   -0.5|     F|      2|
|    0.0|     F|      3|
|    3.0|     M|      5|
+-------+------+-------+



In [12]:
df.na.drop(subset=['gender']).show()

+-------+------+-------+
|balance|gender|user_id|
+-------+------+-------+
|    1.0|     M|      1|
|   -0.5|     F|      2|
|    0.0|     F|      3|
|    3.0|     M|      5|
+-------+------+-------+



In [13]:
df.na.fill({'gender': 'U', 'balance': 0}).show()

+-------+------+-------+
|balance|gender|user_id|
+-------+------+-------+
|   10.0|     U|      0|
|    1.0|     M|      1|
|   -0.5|     F|      2|
|    0.0|     F|      3|
|    5.0|     U|      4|
|    3.0|     M|      5|
+-------+------+-------+



In [14]:
# 聚合函数
df.na.fill({
    'gender': 'U',
    'balance': 0
}).groupBy('gender').avg('balance').show()

+------+------------+
|gender|avg(balance)|
+------+------------+
|     F|       -0.25|
|     M|         2.0|
|     U|         7.5|
+------+------------+



In [15]:
# 转换
pdf = df.toPandas()

In [16]:
pdf.head()

,balance,gender,user_id
0,10.0,None,0
1,1.0,M,1
2,-0.5,F,2
3,0.0,F,3
4,5.0,None,4


In [17]:
df.registerTempTable('users')

In [18]:
table = sqlc.table('users')

In [19]:
type(sqlc.table('users'))

pyspark.sql.dataframe.DataFrame

In [20]:
sqlc.sql(
    'select gender, AVG(balance) from users where gender is not null group by gender'
).show()

+------+------------+
|gender|avg(balance)|
+------+------------+
|     F|       -0.25|
|     M|         2.0|
+------+------------+



In [21]:
table.show()

+-------+------+-------+
|balance|gender|user_id|
+-------+------+-------+
|   10.0|  null|      0|
|    1.0|     M|      1|
|   -0.5|     F|      2|
|    0.0|     F|      3|
|    5.0|  null|      4|
|    3.0|     M|      5|
+-------+------+-------+



In [22]:
table.collect()

[Row(balance=10.0, gender=None, user_id=0),
 Row(balance=1.0, gender='M', user_id=1),
 Row(balance=-0.5, gender='F', user_id=2),
 Row(balance=0.0, gender='F', user_id=3),
 Row(balance=5.0, gender=None, user_id=4),
 Row(balance=3.0, gender='M', user_id=5)]

In [23]:
a_row = sqlc.sql('select * from users').first()
print(a_row)

Row(balance=10.0, gender=None, user_id=0)


In [24]:
print(a_row['balance'])
print(a_row.gender)

10.0
None


In [25]:
a_row.asDict()

{'balance': 10.0, 'gender': None, 'user_id': 0}

In [26]:
# 保存
df.na.drop().write.save(
    'file:///D:/Projects/python_projects/big_data/user_demo2.csv', format='csv')

In [27]:
# 保存
df.na.drop().write.save(
    'file:///D:/Projects/python_projects/big_data/user_demo2.json',
    format='json')

In [28]:
# 保存
df.na.drop().write.save(
    'file:///D:/Projects/python_projects/big_data/user_demo2.parquet',
    format='parquet')

In [29]:
# 读取
sqlc.sql(
    'select * from csv.`file:///D:/Projects/python_projects/big_data/user_demo2.csv`'
).show()

+----+---+---+
| _c0|_c1|_c2|
+----+---+---+
| 1.0|  M|  1|
|-0.5|  F|  2|
| 0.0|  F|  3|
| 3.0|  M|  5|
+----+---+---+



In [30]:
# 读取
sqlc.sql(
    'select * from json.`file:///D:/Projects/python_projects/big_data/user_demo2.json`'
).show()

+-------+------+-------+
|balance|gender|user_id|
+-------+------+-------+
|    1.0|     M|      1|
|   -0.5|     F|      2|
|    0.0|     F|      3|
|    3.0|     M|      5|
+-------+------+-------+



In [31]:
# 读取
sqlc.sql(
    'select * from parquet.`file:///D:/Projects/python_projects/big_data/user_demo2.parquet`'
).show()

+-------+------+-------+
|balance|gender|user_id|
+-------+------+-------+
|    1.0|     M|      1|
|   -0.5|     F|      2|
|    0.0|     F|      3|
|    3.0|     M|      5|
+-------+------+-------+



In [32]:
# 创建数据表
from pyspark.sql import Row
rdd_gender = sc.parallelize([
    Row(short_gender='M', long_gender='Male'),
    Row(short_gender='F', long_gender='Female')
])
sqlc.createDataFrame(rdd_gender).registerTempTable('gender_maps')

In [33]:
sqlc.table('gender_maps').show()

+-----------+------------+
|long_gender|short_gender|
+-----------+------------+
|       Male|           M|
|     Female|           F|
+-----------+------------+



In [34]:
# 联合查询
sqlc.sql('''
select balance, long_gender, user_id
from users join gender_maps
on gender=short_gender
''').show()

+-------+-----------+-------+
|balance|long_gender|user_id|
+-------+-----------+-------+
|    3.0|       Male|      5|
|    1.0|       Male|      1|
|    0.0|     Female|      3|
|   -0.5|     Female|      2|
+-------+-----------+-------+



In [35]:
sqlc.tableNames()

['gender_maps', 'users']

In [36]:
# 删除内存的数据表
for table in sqlc.tableNames():
    sqlc.dropTempTable(table)